In [2]:
import numpy as np

## Generating random distance

In [3]:
def generate_distances(num_cities):
    distances = np.zeros(shape=(num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j and distances[i, j] == 0:
                distances[i, j] = distances[j, i] = np.random.randint(3, 51)
    return distances

In [116]:
class AntColony:
    def __init__(self, distances, num_ants, num_iterations, evaporation_rate=0.5, alpha=1, beta=2):
        self.distances = distances
        self.num_cities = len(distances)
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.pheromones = np.ones_like(distances)
    
    def run(self):
        shortest_path = None
        shortest_distance = float('inf')

        for _ in range(self.num_iterations):
            paths = self.generate_paths()
            self.update_pheromones(paths)
            for path in paths:
                path.append(path[0])
                distance = self.calculate_distance(path)
                if distance < shortest_distance:
                    shortest_distance = distance
                    shortest_path = path
        return shortest_path, shortest_distance
   
    def update_pheromones(self, paths):
        pheromone_delta = np.zeros_like(self.pheromones)
        best_path, shortest_distance, longest_distance = self.find_best_worst_path(paths)
        
        for i in range(len(best_path) - 1):
            cur_city = best_path[i]
            next_city = best_path[i + 1]
            pheromone_delta[cur_city, next_city] += shortest_distance / longest_distance
            pheromone_delta[next_city, cur_city] += shortest_distance / longest_distance
        self.pheromones = (1 - self.evaporation_rate) * self.pheromones + pheromone_delta
    
    def find_best_worst_path(self, paths):
        best = None
        shortest = float('inf')
        longest = float('-inf')
        for path in paths:
            distance = self.calculate_distance(path)
            if distance < shortest:
                best = path
                shortest = distance
            longest = max(longest, distance)
        
        return best, shortest, longest
        
    def calculate_distance(self, path):
        distance = 0
        for i in range(len(path) - 1):
            current_city = path[i]
            next_city = path[i + 1]
            distance += self.distances[current_city, next_city]
        return distance
        
    def generate_paths(self):
        paths = []
        for _ in range(self.num_ants):
            path = self.generate_path()
            paths.append(path)
        return paths
        
    def generate_path(self, start_city=0):
        # path = [start_city] 
        path = [np.random.randint(0, self.num_cities)]
        # Add cities according to probability
        while len(path) < self.num_cities:
            next_city = self.choose_next_city(path)
            path.append(next_city)
        return path
        
    def choose_next_city(self, path):
        cur_city = path[-1]
        unvisited_cities = [city for city in range(self.num_cities) if city not in path]
        prob = [self.calculate_prob(cur_city, city) for city in unvisited_cities]
        total_prob = sum(prob)
        normalized_prob = [p / total_prob for p in prob]
        next_city = np.random.choice(unvisited_cities, p=normalized_prob)
        return next_city
        
    def calculate_prob(self, cur_city, next_city):
        pheromone = self.pheromones[cur_city, next_city]
        distance = self.distances[cur_city, next_city]
        total = 0
        for city in range(self.num_cities):
            if city != cur_city:
                total += (self.pheromones[cur_city, city] ** self.alpha) * (1 / self.distances[cur_city, city] ** self.beta)
        prob = (pheromone ** self.alpha) * (1 / distance ** self.beta) / total
        return prob

In [117]:
num_iterations = 50
distances_10 = generate_distances(10)
distances = np.zeros(shape=(4, 4))
distances[0, 1] = distances[1, 0] = 10
distances[0, 2] = distances[2, 0] = 15
distances[0, 3] = distances[3, 0] = 20
distances[1, 3] = distances[3, 1] = 25
distances[2, 3] = distances[3, 2] = 30
distances[1, 2] = distances[2, 1] = 35

num_ants = [1, 5, 10, 20]
for ants in num_ants:
    colony = AntColony(distances_10, ants, num_iterations)
    shortest_path, shortest_distance = colony.run()
    print(shortest_path, shortest_distance)

[2, 8, 3, 6, 1, 0, 5, 4, 7, 9, 2] 124.0
[3, 5, 0, 1, 6, 8, 2, 9, 4, 7, 3] 106.0
[5, 0, 1, 6, 8, 2, 9, 4, 7, 3, 5] 106.0
[8, 6, 1, 0, 5, 3, 7, 4, 9, 2, 8] 106.0


In [120]:
num_iterations = 50
distances_20 = generate_distances(20)
num_ants = [1, 5, 10, 20]
for ants in num_ants:
    colony = AntColony(distances_20, ants, num_iterations)
    shortest_path, shortest_distance = colony.run()
    print(shortest_path, shortest_distance)

[10, 0, 11, 17, 18, 8, 7, 2, 4, 3, 12, 6, 9, 14, 5, 1, 16, 13, 15, 19, 10] 230.0
[1, 11, 17, 18, 12, 6, 9, 5, 14, 2, 4, 3, 8, 7, 0, 10, 19, 15, 13, 16, 1] 183.0
[13, 15, 19, 6, 9, 5, 14, 1, 11, 17, 12, 18, 8, 7, 0, 10, 16, 2, 4, 3, 13] 182.0
[11, 17, 18, 12, 6, 9, 5, 0, 7, 8, 3, 4, 2, 14, 1, 16, 10, 19, 15, 13, 11] 182.0
